In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Dataset

In [3]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2),
    transforms.RandomRotation(degrees=20),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=256, shuffle=True, drop_last = True)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=256, shuffle=False, drop_last = True)

100%|██████████| 170498071/170498071 [00:03<00:00, 49087756.23it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


# Model

In [4]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
        )

        self.classifier = nn.Sequential(
            nn.Linear(4 * 4 * 256, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, 4 * 4 * 256)
        x = self.classifier(x)
        return x

# Train Function

In [5]:
def train(criterion, optimizer, scheduler, model, num_epochs, device):
    loss_history = []
    test_acc_history = []

    for epoch in range(num_epochs):
        model.train()
        loss_per_epoch = 0

        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            loss_per_epoch += loss.item()

        scheduler.step()

        correct = 0
        total = 0
        model.eval()

        with torch.no_grad():
            for images, labels in testloader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        epoch_loss = loss_per_epoch / len(trainloader)
        epoch_acc = correct / total

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Test Accuracy: {epoch_acc:.4f}")
        loss_history.append(epoch_loss)
        test_acc_history.append(epoch_acc)

    return loss_history, test_acc_history

In [6]:
model = VGG()
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)

num_epochs = 30
loss_history, test_acc_history = train(criterion, optimizer, scheduler, model, num_epochs, DEVICE)

epoch: 0, loss_per_epoch: 368.2815246582031, test_accuracy: 0.37049278846153844
epoch: 1, loss_per_epoch: 281.6642761230469, test_accuracy: 0.5155248397435898
epoch: 2, loss_per_epoch: 230.44444274902344, test_accuracy: 0.5955528846153846
epoch: 3, loss_per_epoch: 200.229248046875, test_accuracy: 0.655448717948718
epoch: 4, loss_per_epoch: 175.80667114257812, test_accuracy: 0.6715745192307693
epoch: 5, loss_per_epoch: 156.90948486328125, test_accuracy: 0.7216546474358975
epoch: 6, loss_per_epoch: 145.3646240234375, test_accuracy: 0.7233573717948718
epoch: 7, loss_per_epoch: 130.2404022216797, test_accuracy: 0.7459935897435898
epoch: 8, loss_per_epoch: 120.7616958618164, test_accuracy: 0.7686298076923077
epoch: 9, loss_per_epoch: 112.94281768798828, test_accuracy: 0.7735376602564102
epoch: 10, loss_per_epoch: 104.33758544921875, test_accuracy: 0.7728365384615384
epoch: 11, loss_per_epoch: 98.05274200439453, test_accuracy: 0.7841546474358975
epoch: 12, loss_per_epoch: 90.84552764892578, 